In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tkinter as tk
import os
from tkinter import ttk, messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.preprocessing import LabelEncoder

In [23]:
# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

for dirname, _, filenames in os.walk('data_base'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data_base\US_Accidents_March23.csv


In [11]:
# Cargar el conjunto de datos
data = pd.read_csv('data_base/US_Accidents_March23.csv', usecols=['City', 'Start_Time', 'Severity', 'Weather_Condition'])

In [ ]:
# Retorna las primeras 5 filas del DataFrame
data[['Weather_Condition', 'City', 'Start_Time', 'Severity']].head()
#data.head()

,Weather_Condition,City,Start_Time,Severity
0,Light Rain,Dayton,2016-02-08 05:46:00,3
1,Light Rain,Reynoldsburg,2016-02-08 06:07:59,2
2,Overcast,Williamsburg,2016-02-08 06:49:27,2
3,Mostly Cloudy,Dayton,2016-02-08 07:23:34,3
4,Mostly Cloudy,Dayton,2016-02-08 07:39:07,2


In [24]:
# Funciones matemáticas
def gauss_seidel(a, b, x0=None, tol=1e-6, max_iter=100):
    n = len(b)
    x = x0 or [0] * n
    for _ in range(max_iter):
        x_new = x[:]
        for i in range(n):
            s1 = sum(a[i][j] * x_new[j] for j in range(i))
            s2 = sum(a[i][j] * x[j] for j in range(i+1, n))
            x_new[i] = (b[i] - s1 - s2) / a[i][i]
        if max(abs(x_new[i] - x[i]) for i in range(n)) < tol:
            return x_new
        x = x_new
    return x

def ajustar_minimos_cuadrados(X, Y):
    A = [[sum(x**p for x in X) for p in range(3)] for _ in range(3)]
    for i in range(3):
        for j in range(3):
            A[i][j] = sum(x**(i + j) for x in X)
    b = [sum(y * (x**i) for x, y in zip(X, Y)) for i in range(3)]
    coeficientes = gauss_seidel(A, b)
    return coeficientes  # [a0, a1, a2] para ax² + bx + c

In [31]:
CIUDADES_VALIDAS = [
    'Los Angeles', 'New York', 'Dayton', 'Chicago', 'Atlanta', 'Philadelphia',
    'San Antonio', 'San Diego', 'Dallas', 'San Jose', 'Austin', 'Miami',
    'Fort Worth', 'Columbus', 'San Francisco', 'Orlando', 'Indianapolis',
    'Seattle', 'Denver', 'Washington', 'Nashville', 'Oklahoma City', 'Boston',
    'El Paso', 'Portland', 'Las Vegas', 'Detroit', 'Memphis', 'Louisville',
    'Milwaukee', 'Albuquerque', 'Tucson'
]

def cargar_datos(data):
    try:
        # Filtrar por ciudades válidas
        data = data[data['City'].isin(CIUDADES_VALIDAS)].dropna()

        # Calcular "Momento" (día o noche) a partir de la hora
        data['Moment'] = data['Start_Time'].apply(
            lambda x: 'Día' if 6 <= int(x[11:13]) <= 18 else 'Noche')

        # Convertir gravedad a entero (por si viene como float)
        data['Severity'] = data['Severity'].astype(int)

        # Seleccionar columnas necesarias
        data_final = data[['City', 'Weather_Condition', 'Moment', 'Severity']]

        # Convertir a lista de diccionarios si deseas mantener la compatibilidad con tu código anterior
        return data_final.to_dict('records')

    except Exception as e:
        print("Error cargando el archivo:", e)
        return []

def predecir_gravedad(ciudad, clima, momento, datos):
    filtrados = [d for d in datos if
                 d['City'] == ciudad and
                 d['Weather_Condition'] == clima and
                 d['Moment'] == momento]

    if not filtrados or len(filtrados) < 3:
        return None, []

    # Crear eje X como índice simple (0, 1, 2, ...)
    X = list(range(len(filtrados)))
    Y = [d['Severity'] for d in filtrados]

    # Obtener coeficientes por mínimos cuadrados + Gauss-Seidel
    coef = ajustar_minimos_cuadrados(X, Y)

    # Predecir gravedad para siguiente punto (len(X)) usando ax² + bx + c
    x_pred = len(X)
    prediccion = coef[0] + coef[1]*x_pred + coef[2]*(x_pred**2)
    return prediccion, (X, Y, coef)

In [32]:
def crear_interfaz(datos):
    def ejecutar_prediccion():
        ciudad = ciudad_combo.get()
        clima = clima_combo.get()
        hora = int(hora_spinbox.get())

        if not ciudad or not clima:
            messagebox.showerror("Error", "Por favor seleccione ciudad y clima.")
            return

        momento = 'Día' if 6 <= hora <= 18 else 'Noche'

        pred, (X, Y, coef) = predecir_gravedad(ciudad, clima, momento, datos)
        if pred is None:
            resultado_var.set("No hay suficientes datos.")
            return

        resultado_var.set(f"Predicción de gravedad: {pred:.2f} ({momento})")

        # Graficar
        plt.figure(figsize=(6,4))
        plt.scatter(X, Y, label="Datos Reales", color='blue')
        x_curve = np.linspace(0, len(X)+2, 100)
        y_curve = coef[0] + coef[1]*x_curve + coef[2]*x_curve**2
        plt.plot(x_curve, y_curve, label="Curva Ajustada", color='red')
        plt.title(f"{ciudad} - {clima} - {momento}")
        plt.xlabel("Índice")
        plt.ylabel("Gravedad")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    root = tk.Tk()
    root.title("Predicción de Gravedad de Accidentes")

    # Ciudad
    tk.Label(root, text="Ciudad:").grid(row=0, column=0, padx=5, pady=5)
    ciudad_combo = ttk.Combobox(root, values=CIUDADES_VALIDAS, width=20)
    ciudad_combo.grid(row=0, column=1, padx=5, pady=5)

    # Clima
    condiciones_unicas = sorted(set([d['Weather_Condition'] for d in datos]))
    tk.Label(root, text="Clima:").grid(row=1, column=0, padx=5, pady=5)
    clima_combo = ttk.Combobox(root, values=condiciones_unicas, width=20)
    clima_combo.grid(row=1, column=1, padx=5, pady=5)

    # Hora
    tk.Label(root, text="Hora (0-23):").grid(row=2, column=0, padx=5, pady=5)
    hora_spinbox = tk.Spinbox(root, from_=0, to=23, width=5)
    hora_spinbox.grid(row=2, column=1, padx=5, pady=5, sticky='w')

    # Botón
    tk.Button(root, text="Predecir gravedad", command=ejecutar_prediccion).grid(row=3, column=0, columnspan=2, pady=10)

    # Resultado
    resultado_var = tk.StringVar()
    tk.Label(root, textvariable=resultado_var, font=("Arial", 12, "bold")).grid(row=4, column=0, columnspan=2, pady=5)

    root.mainloop()

# ------------------ Ejecución ------------------
if __name__ == "__main__":
    datos = cargar_datos(data)
    crear_interfaz(datos)

In [6]:
data = pd.read_csv('data_base/US_Accidents_March23.csv', usecols=['City'])

# Obtener ciudades únicas (sin duplicados), ordenadas alfabéticamente
ciudades = sorted(data['City'].dropna().unique())

# Mostrar todas las ciudades
for ciudad in ciudades:
    print(ciudad)

Aaronsburg
Abbeville
Abbotsford
Abbott
Abbottstown
Aberdeen
Aberdeen Proving Ground
Abernathy
Abilene
Abingdon
Abington
Abiquiu
Abita Springs
Abrams
Absarokee
Absecon
Acampo
Accident
Accokeek
Accomac
Accord
Ackerman
Ackley
Ackworth
Acme
Acoma
Acosta
Acra
Acton
Acushnet
Acworth
Ada
Adah
Adair
Adair Village
Adairsville
Adairville
Adams
Adams Center
Adams Run
Adamsburg
Adamstown
Adamsville
Addis
Addison
Addy
Addyston
Adel
Adelanto
Adell
Adelphi
Adena
Adger
Adin
Adkins
Adolphus
Adrian
Advance
Afton
Agate
Agawam
Agency
Agoura Hills
Aguanga
Aguila
Aguilar
Ahoskie
Ahwahnee
Aiken
Ailey
Ainsworth
Airmont
Airville
Airway Heights
Aitkin
Ajo
Akeley
Akron
Alabaster
Alachua
Alameda
Alamo
Alamogordo
Alamosa
Alanson
Alapaha
Alba
Albany
Albemarle
Albers
Albert City
Albert Lea
Alberta
Alberton
Albertson
Albertville
Albia
Albin
Albion
Albrightsville
Albuquerque
Alburgh
Alburtis
Alcoa
Alcolu
Alcova
Alcove
Alda
Aldan
Alden
Alder
Alder Creek
Alderpoint
Alderson
Aldie
Aldrich
Aledo
Alexander
Alexander City
A